In [1]:
import os
import sys
from pathlib import Path
import cv2
import numpy as np
import re
from typing import Dict, Tuple, Optional


def setup_project_path():
    current = Path.cwd()
    while not (current / 'craft').exists():
        current = current.parent
    return current
project_root = setup_project_path()
sys.path.insert(0, str(project_root))

In [2]:
def imwrite_unicode(path, img):
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True

In [3]:
def imread_unicode(path, flags=cv2.IMREAD_COLOR):
    try:
        with open(path, "rb") as f:
            data = f.read()
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e)
        return None


In [4]:
_CORNER_RE = re.compile(r"^(TL|TR|BR|BL)\s*=\s*\(\s*(-?\d+)\s*,\s*(-?\d+)\s*\)\s*$")
_FILE_RE   = re.compile(r"^file\s*=\s*(.+?)\s*$")

def parse_bbox_txt(txt_path: Path) -> Dict[str, Tuple[int,int,int,int]]:
    mapping: Dict[str, Tuple[int,int,int,int]] = {}

    cur_file: Optional[str] = None
    corners: Dict[str, Tuple[int,int]] = {}

    def flush():
        nonlocal cur_file, corners
        if cur_file is None:
            return
        if all(k in corners for k in ("TL","TR","BR","BL")):
            xs = [corners["TL"][0], corners["TR"][0], corners["BR"][0], corners["BL"][0]]
            ys = [corners["TL"][1], corners["TR"][1], corners["BR"][1], corners["BL"][1]]
            x0, x1 = int(min(xs)), int(max(xs))
            y0, y1 = int(min(ys)), int(max(ys))
            mapping[cur_file] = (x0, y0, x1, y1)
        cur_file = None
        corners = {}

    with open(txt_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()

            if not line:
                continue

            m = _FILE_RE.match(line)
            if m:
                flush()
                cur_file = m.group(1)
                corners = {}
                continue

            if "INVALID" in line.upper():
                flush()
                continue

            m = _CORNER_RE.match(line)
            if m and cur_file is not None:
                key = m.group(1)
                x = int(m.group(2))
                y = int(m.group(3))
                corners[key] = (x, y)

    flush()
    return mapping

In [5]:
def draw_bbox_xyxy(img: np.ndarray, xyxy: Tuple[int,int,int,int], thickness: int = 2, pad: int = 0) -> np.ndarray:
    if img is None:
        return None

    if img.ndim == 2:
        vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    elif img.ndim == 3 and img.shape[2] == 4:
        vis = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    else:
        vis = img.copy()

    h, w = vis.shape[:2]
    x0, y0, x1, y1 = xyxy

    x0 = max(0, x0 - pad)
    y0 = max(0, y0 - pad)
    x1 = min(w - 1, x1 + pad)
    y1 = min(h - 1, y1 + pad)

    cv2.rectangle(vis, (x0, y0), (x1, y1), (0, 0, 255), thickness)
    return vis


In [6]:
def visualize_from_txt_batch(
    images_root: Path,
    txt_root: Path,
    out_root: Path,
    thickness: int = 2,
    pad: int = 0,
    read_flags: int = cv2.IMREAD_UNCHANGED,
    out_ext: str = ".png",
    strict_missing: bool = False,
):
    images_root = Path(images_root)
    txt_root = Path(txt_root)
    out_root = Path(out_root)
    out_root.mkdir(parents=True, exist_ok=True)

    txt_files = sorted([p for p in txt_root.iterdir() if p.is_file() and p.suffix.lower() == ".txt"])

    for txt_path in txt_files:
        folder_name = txt_path.stem
        folder_img_dir = images_root / folder_name
        if not folder_img_dir.exists():
            msg = f"[WARN] image folder not found: {folder_img_dir}"
            if strict_missing:
                raise FileNotFoundError(msg)
            print(msg)
            continue

        file_to_xyxy = parse_bbox_txt(txt_path)

        out_dir = out_root / folder_name
        out_dir.mkdir(parents=True, exist_ok=True)

        for fname, xyxy in file_to_xyxy.items():
            img_path = folder_img_dir / fname
            if not img_path.exists():
                msg = f"[WARN] image missing for bbox entry: {img_path}"
                if strict_missing:
                    raise FileNotFoundError(msg)
                print(msg)
                continue

            img = imread_unicode(img_path, flags=read_flags)
            if img is None:
                print("[WARN] failed to read:", img_path)
                continue

            vis = draw_bbox_xyxy(img, xyxy, thickness=thickness, pad=pad)
            out_path = out_dir / (Path(fname).stem + out_ext)
            imwrite_unicode(out_path, vis)

    print("DONE")

In [7]:
images_root = project_root / "characters"
txt_root    = project_root /  "results" / "char_bbox"
out_root    = project_root / "visualize" / "visualize_3"

visualize_from_txt_batch(
    images_root=images_root,
    txt_root=txt_root,
    out_root=out_root,
    thickness=2,
    pad=0,
    out_ext=".png",
    strict_missing=False,
)


DONE
